In [1]:
%reload_ext autoreload
%autoreload 2
%env AD_MAX_ITEMS=1000
%env AD_SHOW_FULL_STRINGS=1
from torch_snippets import *


env: AD_MAX_ITEMS=1000
env: AD_SHOW_FULL_STRINGS=1


In [2]:
# import all csv files as duckdb connection
root = P("/Users/yeshwanth/Code/Divami/foundry/nl2sql/data/ipl 2008-2024")
import duckdb, os

database = root / "data.duckdb"

if not database.exists():
    con = duckdb.connect(database=database)
    for filename in os.listdir(root):
        if filename.endswith(".csv"):
            table_name = filename[:-4]
            con.execute(
                f"CREATE TABLE {table_name} AS SELECT * FROM read_csv_auto('{root}/{filename}')"
            )


In [3]:
from functools import lru_cache


class DuckDBSchemaExplorer:
    def __init__(self, database=None):
        self.database = database
        self.conn = duckdb.connect(database=database)
        self.tables = AD({
            t[0]: self.describe(t[0])
            for t in self.conn.execute("SHOW TABLES;").fetchall()
        })

    @lru_cache
    def unique_values_in_column(self, table_name, column_name):
        nunique = self.conn.execute(
            f"SELECT count(DISTINCT {column_name}) FROM {table_name};"
        ).fetchall()[0][0]
        unique = None
        if nunique < 30:
            unique = [
                v[0]
                for v in self.conn.execute(
                    f"SELECT DISTINCT {column_name} FROM {table_name};"
                ).fetchall()
            ]
        return nunique, unique

    def describe(self, table_name):
        df = pd.DataFrame(
            self.conn.execute(f"DESCRIBE {table_name};").fetchall(),
            columns=["column_name", "data_type", "nullable", "key", "default", "extra"],
        )
        df["unique_values"] = df["column_name"].apply(
            lambda col: self.unique_values_in_column(table_name, col)[0]
        )
        df["sample_values"] = df["column_name"].apply(
            lambda col: self.unique_values_in_column(table_name, col)[1]
        )
        return df.to_dict(orient="records")

    def query(self, query: str):
        return self.conn.execute(query).fetchall()

In [4]:
# create a duckdb connection
schema_explorer = DuckDBSchemaExplorer(database)
system_prompt = str(schema_explorer.tables)

In [5]:
from pydantic_ai import Agent, RunContext
from dataclasses import dataclass
from pydantic import BaseModel


@dataclass
class Deps:
    schema_explorer: DuckDBSchemaExplorer


class Output(BaseModel):
    result: str
    sql_query: str


agent = Agent(
    "google-gla:gemini-2.5-flash",
    system_prompt=system_prompt,
    deps_type=Deps,
    output_type=Output,
)


@agent.tool
def run_query(ctx: RunContext, query: str) -> str:
    return ctx.deps.schema_explorer.query(query)


In [6]:
async def main(query):
    x = await agent.run(query, deps=Deps(schema_explorer))
    return AD(x.output.model_dump())

In [ ]:
await main("Who scored most runs in 2016")  # 2008 - 2024


```↯ AttrDict ↯
result - V Kohli scored 973 runs in 2016, which is the most runs scored in that season. (🏷️ str)
sql_query - SELECT d.batter, SUM(d.batsman_runs) AS total_runs FROM deliveries d JOIN matches m ON d.match_id = m.id WHERE m.season = '2016' GROUP BY d.batter ORDER BY total_runs DESC LIMIT 1 (🏷️ str)

```

In [27]:
await main("Who took most wickets for a single team?")


```↯ AttrDict ↯
result - SP Narine took the most wickets for a single team, with 200 wickets for Kolkata Knight Riders. (🏷️ str)
sql_query - SELECT bowler, bowling_team, COUNT(is_wicket) AS wickets FROM deliveries WHERE is_wicket = 1 GROUP BY bowler, bowling_team ORDER BY wickets DESC LIMIT 1 (🏷️ str)

```

In [7]:
await main("Who scored the most runs in 2010?")


```↯ AttrDict ↯
result - SR Tendulkar scored the most runs in the 2010 season with 618 runs. (🏷️ str)
sql_query - SELECT d.batter, SUM(d.batsman_runs) AS total_runs FROM deliveries d JOIN matches m ON d.match_id = m.id WHERE m.season = '2009/10' GROUP BY d.batter ORDER BY total_runs DESC LIMIT 1 (🏷️ str)

```

In [8]:
await main(
    "List the number of matches every team lost with either fewer than 5 runs margin or less than 3 wickets margin"
)


```↯ AttrDict ↯
result - ↓
  ```
  Here is a list of the number of matches each team lost with either fewer than 5 runs margin or less than 3 wickets margin:
  
  Mumbai Indians: 9 matches
  Kolkata Knight Riders: 9 matches
  Chennai Super Kings: 6 matches
  Rajasthan Royals: 5 matches
  Sunrisers Hyderabad: 5 matches
  Delhi Daredevils: 5 matches
  Kings XI Punjab: 5 matches
  Royal Challengers Bangalore: 3 matches
  Deccan Chargers: 3 matches
  Rising Pune Supergiants: 2 matches
  Lucknow Super Giants: 2 matches
  Punjab Kings: 2 matches
  Rising Pune Supergiant: 1 match
  Royal Challengers Bengaluru: 1 match
  Pune Warriors: 1 match
  Delhi Capitals: 1 match
  Gujarat Titans: 1 match
  Gujarat Lions: 1 match
  ``` (🏷️ Multiline str)
sql_query - SELECT CASE WHEN T1.team1 = T1.winner THEN T1.team2 ELSE T1.team1 END AS losing_team, COUNT(*) AS matches_lost FROM matches AS T1 WHERE (T1.result = 'runs' AND CAST(T1.result_margin AS INT) < 5) OR (T1.result = 'wickets' AND CAST(T1.result_m

In [ ]:
await main("Who are the batsmen who scored a century in IPL final")


```↯ AttrDict ↯
result - SR Watson and WP Saha are the batsmen who scored a century in an IPL final. (🏷️ str)
sql_query - SELECT DISTINCT d.batter FROM deliveries AS d INNER JOIN matches AS m ON d.match_id = m.id WHERE m.match_type = 'Final' GROUP BY d.match_id, d.batter HAVING SUM(d.batsman_runs) >= 100; (🏷️ str)

```

In [24]:
await main(
    "I want the best all rounders in IPL. Select those who bowled at least 50 overs and scored at least 500 runs. Multiply their inverse (reciprocal of) bowling economy and the strike rate and use that as criteria to measure as all rounder performance"
)


```↯ AttrDict ↯
result - ↓
  ```
  Here are the best all-rounders in IPL based on your criteria (at least 50 overs bowled, at least 500 runs scored, and ranked by (1 / bowling economy) * batting strike rate):
  
  1. SP Narine - All-rounder performance: 23.06 (Runs: 1534, Batting SR: 155.89, Overs: 691.0, Bowling Economy: 6.76)
  2. Rashid Khan - All-rounder performance: 22.22 (Runs: 545, Batting SR: 153.52, Overs: 483.5, Bowling Economy: 6.91)
  3. MM Ali - All-rounder performance: 19.52 (Runs: 1162, Batting SR: 136.87, Overs: 128.33, Bowling Economy: 7.01)
  4. CH Morris - All-rounder performance: 18.90 (Runs: 618, Batting SR: 149.64, Overs: 300.17, Bowling Economy: 7.92)
  5. YK Pathan - All-rounder performance: 18.88 (Runs: 3222, Batting SR: 138.05, Overs: 197.33, Bowling Economy: 7.31)
  6. Harbhajan Singh - All-rounder performance: 18.61 (Runs: 833, Batting SR: 130.97, Overs: 582.67, Bowling Economy: 7.04)
  7. GJ Maxwell - All-rounder performance: 18.46 (Runs: 2772, Batting SR:

In [ ]:
await main("how many runs did M Muralitharan score?")


```↯ AttrDict ↯
result - M Muralitharan scored 20 runs. (🏷️ str)
sql_query - SELECT SUM(batsman_runs) FROM deliveries WHERE batter = 'M Muralitharan' (🏷️ str)

```

In [ ]:
await main(
    "For allrounders, because they contribute with both bat and ball, it's important to analyze their performance in both aspects. Now, What i want to understand is, if a batsman is creating a heavy impact with the strike rate then its a positive score, and if a bowler is slowing things down with a low economy rate then its a positive score for a bowler. Combining the two in terms of impact, can you rate the top ten allrounders in the history of IPL combining both their batting and bowling impact"
)